In [1]:
import psycopg2
import pandas as pd
import csv

### Create connection and cursor

In [2]:
conn = psycopg2.connect(host="localhost", port = 5432, database="aio_enem", user="nicolagp", password="123")
# Create a cursor object
cur = conn.cursor()

### Execute query

In [3]:
cur.execute("""SELECT * FROM enem_student LIMIT 10""")
query_results = cur.fetchall()
query_results

[(1, 150000001161, 22, 'M', '1', '2', 1),
 (2, 150000001219, 18, 'M', '0', '3', 2),
 (3, 150000002884, 19, 'M', '0', '1', 3),
 (4, 150000002510, 17, 'F', '0', '1', 4),
 (5, 150000000658, 26, 'M', '0', '2', 5),
 (6, 150000000216, 29, 'F', '2', '1', 6),
 (7, 150000000274, 25, 'M', '1', '1', 7),
 (8, 150000000164, 20, 'F', '0', '1', 8),
 (9, 150000000218, 21, 'M', '0', '2', 9),
 (10, 150000000219, 22, 'F', '0', '1', 10)]

### Query from pandas

In [4]:
sample_students = pd.read_sql_query("""SELECT * FROM enem_student ORDER BY RANDOM() LIMIT 1000""", conn)

In [5]:
test_id = sample_students.iloc[0,0]
test_id

1003

In [6]:
qs = pd.read_sql_query("""SELECT * FROM enem_questionstudent WHERE student_id = 36653 ORDER BY RANDOM()""", conn)
qs

In [72]:
questions = pd.read_sql_query("""SELECT * FROM enem_question""", conn)
questions = questions.set_index("id")
questions

,inep_code,language,correct,skill_id
id,,,,
1,53985,NaN,A,30
2,32577,NaN,E,20
3,50212,NaN,A,23
4,45004,NaN,E,27
5,25075,NaN,A,13
...,...,...,...,...
560,29402,NaN,A,26
561,96366,NaN,E,26
562,83860,NaN,A,19


## Test CSV

Get students

In [84]:
q = """
SELECT id
FROM enem_student
ORDER BY RANDOM()
LIMIT 1000;
"""
students = list(pd.read_sql_query(q, conn)["id"].values)
students[:5]

[47038, 12109, 50198, 35074, 4440]

Get questions answered by each student

In [105]:
def get_questions(student_id, conn):
    """
    Returns a dataframe with questions answered by a student
    Needs a student id and a connection to the db
    """
    q = f"""
    SELECT student_id, question_id, letter
    FROM enem_questionstudent
    WHERE student_id = {student_id}
    ORDER BY RANDOM();
    """
    return pd.read_sql_query(q, conn)

get_questions(students[15], conn)

,student_id,question_id,letter
0,34924,179,C
1,34924,91,A
2,34924,166,D
3,34924,25,B
4,34924,112,E
...,...,...,...
175,34924,172,C
176,34924,145,E
177,34924,77,B
178,34924,152,E


Get answer key

In [132]:
def get_answers(conn):
    q = """
    SELECT id, correct, skill_id
    FROM enem_question
    """
    gabarito = pd.read_sql_query(q, conn)
    gabarito = gabarito.set_index("id")
    return gabarito

Add "correct" column for a student

In [143]:
def correct(x):
    """
    given that a 'questions' answer key is defined, it checks wether a student's answer is correct
    """
    return 1 if x[2] == gabarito.loc[x[1], "correct"] else 0

stu = get_questions(students[15], conn)
stu["correct"] = stu.apply(correct, axis=1)
stu

,student_id,question_id,letter,correct
0,34924,74,E,0
1,34924,138,D,1
2,34924,163,D,0
3,34924,162,B,0
4,34924,105,B,0
...,...,...,...,...
175,34924,2,D,0
176,34924,59,C,0
177,34924,90,D,0
178,34924,38,E,0


Skill ids

In [127]:
stu["skill_id"] = gabarito.loc[stu["question_id"], "skill_id"].values
stu

,student_id,question_id,letter,correct,skill_id
0,34924,171,E,0,29
1,34924,24,A,0,13
2,34924,23,B,0,14
3,34924,160,D,0,3
4,34924,165,E,0,27
...,...,...,...,...,...
175,34924,125,A,0,16
176,34924,149,A,0,11
177,34924,8,C,1,18
178,34924,153,A,0,7


## Preprocessing Pipeline

In [144]:
def get_questions(student_id, conn):
    """
    Returns a dataframe with questions answered by a student
    Needs a student id and a connection to the db
    """
    q = f"""
    SELECT student_id, question_id, letter
    FROM enem_questionstudent
    WHERE student_id = {student_id}
    ORDER BY RANDOM();
    """
    return pd.read_sql_query(q, conn)

def get_answers(conn):
    q = """
    SELECT id, correct, skill_id
    FROM enem_question
    """
    gabarito = pd.read_sql_query(q, conn)
    gabarito = gabarito.set_index("id")
    return gabarito

def correct(x):
    """
    given that a 'questions' answer key is defined, it checks wether a student's answer is correct
    """
    return 1 if x[2] == gabarito.loc[x[1], "correct"] else 0

In [155]:
def pipeline(students, conn):
    gabarito = get_answers(conn)
    
    df = pd.DataFrame(columns = ["student_id", "question_id", "letter", "correct", "skill_id"])

    for student in students:
        # get student answers
        stu = get_questions(student, conn)
        
        # handle missing data
        if stu.empty:
            continue
            
        # correct col
        stu["correct"] = stu.apply(correct, axis=1)

        # skill_id col
        stu["skill_id"] = gabarito.loc[stu["question_id"], "skill_id"].values

        # concat df
        df = pd.concat((df, stu), axis=0)
    
    # rename cols
    df = df.rename({"student_id": "user_id", "question_id": "item_id"}, axis=1)
    
    # add timestamp
    df["timestamp"] = 0
    
    # return columns in correct order
    cols = ["user_id", "item_id", "timestamp", "correct", "skill_id"]
    return df.loc[:, cols]

In [158]:
df = pipeline(students, conn)

In [159]:
df

,user_id,item_id,timestamp,correct,skill_id
0,12109,127,0,1,29
1,12109,152,0,0,17
2,12109,143,0,1,15
3,12109,162,0,0,18
4,12109,90,0,0,4
...,...,...,...,...,...
175,33205,148,0,1,8
176,33205,92,0,0,8
177,33205,39,0,1,21
178,33205,105,0,0,3
